<a href="https://colab.research.google.com/github/anatorres09/DE-MRI_Classificacio_CNN/blob/main/Model2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Connectar a Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#Importar llibreries
import os
import cv2
import numpy as np
from glob import glob
import tensorflow as tf
from collections import Counter
from tabulate import tabulate

In [19]:
# Carregar i preprocés d'imatges per a la predicció
ruta_carpeta = '/content/drive/MyDrive/CNN'

# Ruta del model
ruta_model = os.path.join(ruta_carpeta, "model_CNN.h5")

# Carregar el model
model = tf.keras.models.load_model(ruta_model)

ruta_carpeta_test = os.path.join(ruta_carpeta, "test")
redimensionar_imatge = (128, 128)

# Funció per a recollir les imatges
def recollir_imatges(ruta_carpeta, redimensionar_imatge):
    casos = [nom_carpeta for nom_carpeta in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, nom_carpeta))]

    imatges = []
    noms_fitxers = []

    for cas in casos:
        ruta_cas = os.path.join(ruta_carpeta, cas)
        rutes_imatges = glob(os.path.join(ruta_cas, '*.png'))

        for ruta_imatge in rutes_imatges:
            imatge = cv2.imread(ruta_imatge)
            imatge = cv2.cvtColor(imatge, cv2.COLOR_BGR2RGB)
            
            # Redimensionar les imatges
            imatge = cv2.resize(imatge, redimensionar_imatge)

            # Obtener el nombre de la subcarpeta y el nombre de la imagen
            num_cas = os.path.basename(ruta_cas)
            num_tall = os.path.basename(ruta_imatge)
            
            
            imatges.append(imatge)
            noms_fitxers.append(f"{num_cas}.{num_tall}")

    return imatges, noms_fitxers

# Recollir imatges de la carpeta de prova
imatges_test, noms_fitxers = recollir_imatges(ruta_carpeta_test, redimensionar_imatge)

# Convertir la llista d'imatges en una matriu numpy
matriu_imatges_test = np.array(imatges_test, dtype='float32')

print(f"S'han recollit {matriu_imatges_test.shape[0]} imatges en total.")


S'han recollit 334 imatges en total.


In [20]:
# Realitzar la predicció utilitzant el model carregat
predicciones = model.predict(matriu_imatges_test)

# Obtindre les classes predites ('normal' o 'patologic')
classes_predites = np.argmax(predicciones, axis=1)

# Obtindre les probabilitats màximes
probabilidades_maximas = np.max(predicciones, axis=1)

# Calcular umbral dinàmic en funció de la probabilitat màxima
umbral = np.mean(probabilidades_maximas)

# Imprimir les classes predites per a cada imatge en la carpeta de prova
for i in range(matriu_imatges_test.shape[0]):
    nom_fitxer = os.path.basename(noms_fitxers[i])
    classe_predita = classes_predites[i]
   
    if classe_predita == 0:
        classe = 'normal'
    else:
        classe = 'patologic'
    
    print(f"La classe predita per a la imatge {nom_fitxer} és: {classe}")

11/11 [==============================] - 0s 6ms/step
La classe predita per a la imatge 0_Case_113.tall_0.png és: normal
La classe predita per a la imatge 0_Case_113.tall_1.png és: normal
La classe predita per a la imatge 0_Case_113.tall_2.png és: normal
La classe predita per a la imatge 0_Case_113.tall_3.png és: normal
La classe predita per a la imatge 0_Case_113.tall_4.png és: normal
La classe predita per a la imatge 0_Case_113.tall_5.png és: normal
La classe predita per a la imatge 0_Case_113.tall_6.png és: normal
La classe predita per a la imatge 0_Case_113.tall_7.png és: normal
La classe predita per a la imatge 0_Case_137.tall_0.png és: patologic
La classe predita per a la imatge 0_Case_137.tall_1.png és: patologic
La classe predita per a la imatge 0_Case_137.tall_2.png és: patologic
La classe predita per a la imatge 0_Case_137.tall_3.png és: patologic
La classe predita per a la imatge 0_Case_137.tall_4.png és: patologic
La classe predita per a la imatge 0_Case_137.tall_5.png és: p

In [28]:
# Crear un diccionari per a agrupar les classificacions per num_cas
classificacions_agrupades = {}

# Imprimir les classes predites per a cada imatge a la carpeta de prova i agrupar-les per num_cas
for i in range(matriu_imatges_test.shape[0]):
    nom_fitxer = os.path.basename(noms_fitxers[i])
    classe_predita = classes_predites[i]
   
    if classe_predita == 0:
        classe = 'normal'
    else:
        classe = 'patològic'
    
    # Obtindre el num_cas de la imatge
    num_cas = nom_fitxer.split('.')[0]
    
    # Afegir la classificació al diccionari
    if num_cas in classificacions_agrupades:
        classificacions_agrupades[num_cas].append(classe)
    else:
        classificacions_agrupades[num_cas] = [classe]


# Imprimir les classificacions agrupades i verificar si són totes iguals per a considerar-les correctes
for num_cas, classificacions in classificacions_agrupades.items():
    # Verificar si totes les classificacions són iguals
    if all(classificacio == classificacions[0] for classificacio in classificacions):
        print(f"{num_cas}     té classificació:                 {classificacions[0]}")
    else:
        print(f"{num_cas}     té classificacions diferents:     {classificacions}")


# Crear una llista de files per la taula
tabla_filas = []
for num_cas, classificacions in classificacions_agrupades.items():
    if all(classificacio == classificacions[0] for classificacio in classificacions):
        tabla_filas.append([num_cas, classificacions[0]])
    else:
        tabla_filas.append([num_cas, "Diferents"] + classificacions)

# Imprimir la taula
headers = ["num_cas", "classificació"]
tabla = tabulate(tabla_filas, headers, tablefmt="grid")
print(tabla)


0_Case_113     té classificació:                 normal
0_Case_137     té classificació:                 patològic
0_Case_106     té classificacions diferents:     ['normal', 'normal', 'patològic', 'normal', 'normal', 'normal', 'normal', 'normal']
0_Case_114     té classificacions diferents:     ['patològic', 'patològic', 'patològic', 'normal', 'patològic', 'normal', 'patològic']
0_Case_103     té classificació:                 patològic
0_Case_141     té classificació:                 patològic
0_Case_132     té classificació:                 patològic
0_Case_123     té classificació:                 normal
0_Case_150     té classificacions diferents:     ['patològic', 'patològic', 'patològic', 'normal', 'patològic', 'patològic', 'patològic']
0_Case_142     té classificació:                 patològic
0_Case_128     té classificació:                 patològic
0_Case_107     té classificació:                 patològic
0_Case_110     té classificació:                 patològic
0_Case_139